In [1]:
from detectron2.data.detection_utils import read_image
from demo.predictors import VisualizationDemo
from detectron2.config import LazyConfig, instantiate
from pathlib import Path
from detectron2.checkpoint import DetectionCheckpointer
import plotly.express as px
import torch
from torch.profiler import profile, record_function, ProfilerActivity

print(Path.cwd())
assert (Path.cwd() / ".project-root").exists(), "Please run this script from the root of the project"

/Users/dgcnz/development/amsterdam/edge


In [2]:
config_file = "projects/dino_dinov2/configs/COCO/dino_dinov2_b_12ep.py"
image_path = "artifacts/idea.jpg"
CONFIDENCE_THRESHOLD = 0.5
opts = ["model.device=cpu", "train.device=cpu", "train.init_checkpoint=artifacts/model_final.pth"]

In [3]:
cfg = LazyConfig.load(config_file)
cfg = LazyConfig.apply_overrides(cfg, opts)
model = instantiate(cfg.model).eval();
checkpointer = DetectionCheckpointer(model);
checkpointer.load(cfg.train.init_checkpoint);

[08/07 16:50:32 timm backbone]: backbone out_indices: (11,)
[08/07 16:50:32 timm backbone]: backbone out_channels: [768]
[08/07 16:50:32 timm backbone]: backbone out_strides: [16]


/Users/dgcnz/development/amsterdam/edge/.venv/lib/python3.10/site-packages/torch/_functorch/deprecated.py:61: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.vmap is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.vmap instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('vmap', 'torch.vmap')


In [4]:
img = read_image(image_path, format="BGR")
demo = VisualizationDemo(model=model)

In [5]:
predictions, visualized_output = demo.run_on_image(img, CONFIDENCE_THRESHOLD);

/Users/dgcnz/development/amsterdam/edge/.venv/lib/python3.10/site-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3588.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [6]:
with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA]) as prof:
    predictions, visualized_output = demo.run_on_image(img, CONFIDENCE_THRESHOLD);

prof.export_chrome_trace("trace.json")

/Users/dgcnz/development/amsterdam/edge/.venv/lib/python3.10/site-packages/torch/autograd/profiler.py:228: UserWarning: CUDA is not available, disabling CUDA profiling
  warn("CUDA is not available, disabling CUDA profiling")
STAGE:2024-08-07 16:50:35 83844:1008192 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2024-08-07 16:50:37 83844:1008192 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2024-08-07 16:50:37 83844:1008192 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


In [7]:
fig = px.imshow(visualized_output.get_image()[:, :, ::-1])
fig.show()